In [2]:
import pandas as pd
import numpy as np

fewshot_models = ['gemini-1.5-flash', 'gemini-2.0-flash', 'gpt-4o']
rpg_fewshot_data = {}
rpg_cot_data = {}
perceived_empathy_data = {}
epitome_data = {}
empatheticdialogues_data = {}

other_models = ['gemini-1.5-flash', 'gpt-4o']

for model in fewshot_models:
    rpg_fewshot_data[model] = {}
    for i in range(5):
        filename = f'data/Robustness Check Data/llm_rpg_fewshot_{model}_{i}.csv'
        rpg_fewshot_data[model][i] = pd.read_csv(filename, index_col=0)

for model in other_models:
    rpg_cot_data[model] = {}
    perceived_empathy_data[model] = {}
    epitome_data[model] = {}
    empatheticdialogues_data[model] = {}
    for i in range(5):
        cot_filename = f'data/Robustness Check Data/llm_rpg_cot_{model}_{i}.csv'
        rpg_cot_data[model][i] = pd.read_csv(cot_filename)
        rpg_cot_data[model][i].drop(columns = 'Unnamed: 0', inplace = True)

        perceived_empathy_filename = f'data/Robustness Check Data/llm_perceived_empathy_{model}_{i}.csv'
        perceived_empathy_data[model][i] = pd.read_csv(perceived_empathy_filename, index_col=0)

        epitome_filename = f'data/Robustness Check Data/llm_epitome_{model}_{i}.csv'
        epitome_data[model][i] = pd.read_csv(epitome_filename, index_col=0)

        empatheticdialogues_filename = f'data/Robustness Check Data/llm_empatheticdialogues_{model}_{i}.csv' 
        empatheticdialogues_data[model][i] = pd.read_csv(empatheticdialogues_filename, index_col=0)

In [3]:
rpg_0 = pd.read_csv('data/Robustness Check Data/llm_rpg_fewshot_gemini-1.5-flash_0.csv')
rpg_0.head()

,Unnamed: 0,conversation_id,full_conversation,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
0,1,20241031124050492385,Seeker: I'm feeling so discouraged.. I just go...,1,1,1,4,1,5
1,4,20241031124638971163,Seeker: I've been feeling so detached from eve...,2,1,1,2,5,1
2,10,20241031125849757353,"Seeker: So, I just lost my job today. I had a ...",3,2,1,2,1,2
3,11,20241031125810052721,Seeker: I'm feeling so discouraged.. I just go...,2,1,1,2,1,4
4,12,20241031130310482335,"Seeker: So, I just lost my job today. I had a ...",4,4,1,1,1,1


In [4]:
rpg_1 = pd.read_csv('data/Robustness Check Data/llm_rpg_fewshot_gemini-1.5-flash_1.csv')
rpg_1.head()

,Unnamed: 0,conversation_id,full_conversation,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
0,1,20241031124050492385,Seeker: I'm feeling so discouraged.. I just go...,1,1,1,4,1,5
1,4,20241031124638971163,Seeker: I've been feeling so detached from eve...,2,1,1,2,5,1
2,10,20241031125849757353,"Seeker: So, I just lost my job today. I had a ...",3,2,1,2,1,2
3,11,20241031125810052721,Seeker: I'm feeling so discouraged.. I just go...,2,1,1,2,1,4
4,12,20241031130310482335,"Seeker: So, I just lost my job today. I had a ...",4,4,1,1,1,1


In [5]:
rpg_2 = pd.read_csv('data/Robustness Check Data/llm_rpg_fewshot_gemini-1.5-flash_2.csv')
rpg_2.head()

,Unnamed: 0,conversation_id,full_conversation,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
0,1,20241031124050492385,Seeker: I'm feeling so discouraged.. I just go...,1,1,1,4,1,5
1,4,20241031124638971163,Seeker: I've been feeling so detached from eve...,2,1,1,2,5,1
2,10,20241031125849757353,"Seeker: So, I just lost my job today. I had a ...",3,2,1,2,1,2
3,11,20241031125810052721,Seeker: I'm feeling so discouraged.. I just go...,2,1,1,2,1,4
4,12,20241031130310482335,"Seeker: So, I just lost my job today. I had a ...",4,4,1,1,1,1


In [6]:
def order_by_column(df, column):
    return df.sort_values(by=column).reset_index(drop=True)

def score_extraction(df, col):
    scores = pd.DataFrame(columns = ['conversation_id',
                                    'validating_emotions', 
                                    'encouraging_elaboration', 
                                    'demonstrating_understanding',
                                    'advice_giving',
                                    'self_oriented',
                                    'dismissing_emotions'])

    for i in range(len(df)):
        try:
            scores = pd.concat([scores, pd.DataFrame({'conversation_id': df.iloc[i, 0],
                                                    'validating_emotions': [int(str(df.iloc[i, 2])[-1])],
                                                        'encouraging_elaboration': [int(str(df.iloc[i, 3])[-1])],
                                                        'demonstrating_understanding': [int(str(df.iloc[i, 4])[-1])],
                                                        'advice_giving': [int(str(df.iloc[i, 5])[-1])],
                                                        'self_oriented': [int(str(df.iloc[i, 6])[-1])],
                                                        'dismissing_emotions': [int(str(df.iloc[i, 7])[-1])]
                                                        })], ignore_index = True)
            scores = order_by_column(scores, col)
        except ValueError as e:
            print(df.iloc[i, 1])
    return scores

rpg_cot_scores = {}
for model in other_models:
    rpg_cot_scores[model] = {}
    for i in range(5):
        rpg_cot_scores[model][i] = score_extraction(rpg_cot_data[model][i], 'conversation_id')

In [7]:
rpg_fewshot_data['gemini-1.5-flash'][0].head()

,conversation_id,full_conversation,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
Unnamed: 0,,,,,,,,
1,20241031124050492385,Seeker: I'm feeling so discouraged.. I just go...,1,1,1,4,1,5
4,20241031124638971163,Seeker: I've been feeling so detached from eve...,2,1,1,2,5,1
10,20241031125849757353,"Seeker: So, I just lost my job today. I had a ...",3,2,1,2,1,2
11,20241031125810052721,Seeker: I'm feeling so discouraged.. I just go...,2,1,1,2,1,4
12,20241031130310482335,"Seeker: So, I just lost my job today. I had a ...",4,4,1,1,1,1


In [8]:
rpg_fewshot_data['gemini-1.5-flash'][1].head()

,conversation_id,full_conversation,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
Unnamed: 0,,,,,,,,
1,20241031124050492385,Seeker: I'm feeling so discouraged.. I just go...,1,1,1,4,1,5
4,20241031124638971163,Seeker: I've been feeling so detached from eve...,2,1,1,2,5,1
10,20241031125849757353,"Seeker: So, I just lost my job today. I had a ...",3,2,1,2,1,2
11,20241031125810052721,Seeker: I'm feeling so discouraged.. I just go...,2,1,1,2,1,4
12,20241031130310482335,"Seeker: So, I just lost my job today. I had a ...",4,4,1,1,1,1


In [9]:
score_columns = rpg_cot_scores['gemini-1.5-flash'][0].reset_index().columns[2:]
dfs = [rpg_cot_scores[model][0].reset_index(), 
        rpg_cot_scores[model][1].reset_index(), 
        rpg_cot_scores[model][2].reset_index(), 
        rpg_cot_scores[model][3].reset_index(), 
        rpg_cot_scores[model][4].reset_index()]
combined = pd.concat(dfs, ignore_index=True)
#print([combined.groupby('conversation_id')['dismissing_emotions'].min() - combined.groupby('conversation_id')['dismissing_emotions'].max()])
agg_funcs = {}
agg_funcs = {c: ['mean', 'min', 'max'] for c in score_columns}
summary = combined.groupby('conversation_id').agg(agg_funcs)
summary.columns = ['_'.join(col).strip() for col in summary.columns.values]

for col in score_columns:
        summary[f'{col}_range'] = summary[f'{col}_max'] - summary[f'{col}_min']

summary.head()

,validating_emotions_mean,validating_emotions_min,validating_emotions_max,encouraging_elaboration_mean,encouraging_elaboration_min,encouraging_elaboration_max,demonstrating_understanding_mean,demonstrating_understanding_min,demonstrating_understanding_max,advice_giving_mean,...,self_oriented_max,dismissing_emotions_mean,dismissing_emotions_min,dismissing_emotions_max,validating_emotions_range,encouraging_elaboration_range,demonstrating_understanding_range,advice_giving_range,self_oriented_range,dismissing_emotions_range
conversation_id,,,,,,,,,,,,,,,,,,,,,
20241031124050492385,3.0,3,3,2.0,2,2,2.0,2,2,3.2,...,3,2.8,2,3,0,0,0,1,0,1
20241031124638971163,3.0,3,3,3.0,3,3,2.2,2,3,3.0,...,4,2.8,2,3,0,0,1,0,0,1
20241031125810052721,2.0,2,2,2.0,2,2,2.0,2,2,3.0,...,1,2.4,2,3,0,0,0,0,0,1
20241031125849757353,3.0,3,3,3.0,3,3,3.0,3,3,3.0,...,1,2.8,2,3,0,0,0,0,0,1
20241031130310482335,3.0,3,3,3.0,3,3,3.0,3,3,2.0,...,1,2.0,2,2,0,0,0,0,0,0


In [10]:
def score_summary(dataset, col, start, model):
    # Your score dimensions
    score_columns = dataset[model][0].reset_index().columns[start:]
    dfs = [dataset[model][0].reset_index(), 
            dataset[model][1].reset_index(), 
            dataset[model][2].reset_index(), 
            dataset[model][3].reset_index(), 
            dataset[model][4].reset_index()]
    combined = pd.concat(dfs, ignore_index=True)
    agg_funcs = {}
    agg_funcs = {c: ['mean', 'min', 'max'] for c in score_columns}

    summary = combined.groupby(col).agg(agg_funcs)
    for c in score_columns:
        combined[c] = pd.to_numeric(combined[c], errors='coerce')
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]

    for col in score_columns:
            summary[f'{col}_range'] = summary[f'{col}_max'] - summary[f'{col}_min']

    return summary

In [11]:
def rpg_final_summary(model, df, start, type):
    rpg_ranges = pd.DataFrame(columns= ['validating_emotions', 
                                        'encouraging_elaboration', 
                                        'demonstrating_understanding', 
                                        'advice_giving', 'self_oriented', 
                                        'dismissing_emotions'])

    summary = score_summary(df, 'conversation_id', start, model)
    cols = summary.columns[1:]
    summary[cols] = summary[cols].apply(pd.to_numeric, errors='coerce')
    summary = summary.describe()[['validating_emotions_range', 
                                                        'encouraging_elaboration_range', 
                                                        'demonstrating_understanding_range', 
                                                        'advice_giving_range', 
                                                        'self_oriented_range', 
                                                        'dismissing_emotions_range']]
    
    summary = summary.rename(columns={'validating_emotions_range': 'validating_emotions', 
                                                    'encouraging_elaboration_range': 'encouraging_elaboration', 
                                                    'demonstrating_understanding_range': 'demonstrating_understanding', 
                                                    'advice_giving_range': 'advice_giving', 
                                                    'self_oriented_range': 'self_oriented', 
                                                    'dismissing_emotions_range': 'dismissing_emotions'})
    summary = summary.loc[['mean', 'std']]

    rpg_ranges = pd.concat([summary], axis=1).T

    # Rename the rows
    rpg_ranges.rename(columns = {'mean': f'{model}_{type}_mean', 'std': f'{model}_{type}_std'}, inplace= True)
    rpg_ranges = rpg_ranges.transpose()
    return rpg_ranges

In [12]:
rpg_final_summary('gemini-1.5-flash', rpg_cot_scores, 2, 'fewshot')

,validating_emotions,encouraging_elaboration,demonstrating_understanding,advice_giving,self_oriented,dismissing_emotions
gemini-1.5-flash_fewshot_mean,0.100000,0.0,0.220000,0.260000,0.12000,0.320000
gemini-1.5-flash_fewshot_std,0.303046,0.0,0.418452,0.486973,0.38545,0.652781


In [13]:
final_rpg = rpg_final_summary('gemini-1.5-flash', rpg_fewshot_data, 3, 'fewshot')
final_rpg = pd.concat([final_rpg, rpg_final_summary('gpt-4o', rpg_fewshot_data, 3, 'fewshot')])
final_rpg = pd.concat([final_rpg, rpg_final_summary('gemini-1.5-flash', rpg_cot_scores, 2, 'cot')])
final_rpg = pd.concat([final_rpg, rpg_final_summary('gpt-4o', rpg_cot_scores, 2, 'cot')])
final_rpg = pd.concat([final_rpg, rpg_final_summary('gemini-2.0-flash', rpg_fewshot_data, 3, 'fewshot')])
final_rpg.to_csv('rpg_robustness_check.csv')

In [14]:
def perceived_empathy_summary(model):
    ranges = pd.DataFrame(columns= ['m_understood', 
                                        'm_validated', 
                                        'm_affirmed', 
                                        'm_seen', 
                                        'm_accepted', 
                                        'm_care',
                                        'm_emotional',
                                        'm_practical',
                                        'm_effort'])

    summary = score_summary(perceived_empathy_data, 'conversation_id', 2, model)
    cols = summary.columns[1:]
    summary[cols] = summary[cols].apply(pd.to_numeric, errors='coerce')
    summary = summary.describe()[['m_understood_range', 
                                  'm_validated_range',
                                  'm_affirmed_range',
                                  'm_seen_range',
                                  'm_accepted_range',
                                  'm_care_range',
                                  'm_emotional_range',
                                  'm_practical_range',
                                  'm_effort_range']]
    
    summary = summary.rename(columns={'m_understood_range': 'validating_emotions', 
                                                    'm_validated_range': 'm_validated', 
                                                    'm_seen_range': 'm_seen', 
                                                    'm_accepted_range': 'm_accepted', 
                                                    'm_care_range': 'm_care', 
                                                    'm_emotional_range': 'm_emotional',
                                                    'm_practical_range': 'm_practical',
                                                    'm_effort_range': 'm_effort'})
    summary = summary.loc[['mean', 'std']]

    ranges = pd.concat([summary], axis=1).T

    # Rename the rows
    ranges.rename(columns = {'mean': f'{model}_mean', 'std': f'{model}_std'}, inplace= True)
    ranges = ranges.transpose()
    return ranges

In [15]:
final_perceived_empathy = perceived_empathy_summary('gemini-1.5-flash')
final_perceived_empathy = pd.concat([final_perceived_empathy, perceived_empathy_summary('gpt-4o')])
# final_perceived_empathy.to_csv('perceived_empathy_robustness_check.csv')

In [16]:
final_perceived_empathy

,validating_emotions,m_validated,m_affirmed_range,m_seen,m_accepted,m_care,m_emotional,m_practical,m_effort
gemini-1.5-flash_mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gemini-1.5-flash_std,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gpt-4o_mean,0.100000,0.160000,0.140000,0.120000,0.320000,0.120000,0.200000,0.200000,0.080000
gpt-4o_std,0.364216,0.370328,0.404566,0.328261,0.471212,0.328261,0.451754,0.404061,0.274048


In [17]:
def epitome_summary(model):
    ranges = pd.DataFrame(columns= ['level_emotional_reactions', 
                                        'level_explorations', 
                                        'level_interpretations'])

    summary = score_summary(epitome_data, 'rp_id', 3, model)
    cols = summary.columns[1:]
    summary[cols] = summary[cols].apply(pd.to_numeric, errors='coerce')
    summary = summary.describe()[['level_emotional_reactions_range', 
                                  'level_explorations_range',
                                  'level_interpretations_range']]
    
    summary = summary.rename(columns={'level_emotional_reactions_range': 'level_emotional_reactions', 
                                                    'level_explorations_range': 'level_explorations', 
                                                    'level_interpretations_range': 'level_interpretations'})
    summary = summary.loc[['mean', 'std']]

    ranges = pd.concat([summary], axis=1).T

    # Rename the rows
    ranges.rename(columns = {'mean': f'{model}_mean', 'std': f'{model}_std'}, inplace= True)
    ranges = ranges.transpose()
    return ranges

In [18]:
final_epitome_summary = epitome_summary('gemini-1.5-flash')
final_epitome_summary = pd.concat([final_epitome_summary, epitome_summary('gpt-4o')])
# final_epitome_summary.to_csv('epitome_robustness_check.csv')

In [19]:
final_epitome_summary

,level_emotional_reactions,level_explorations,level_interpretations
gemini-1.5-flash_mean,0.000000,0.000000,0.000000
gemini-1.5-flash_std,0.000000,0.000000,0.000000
gpt-4o_mean,0.040000,0.100000,0.160000
gpt-4o_std,0.197949,0.303046,0.370328


In [20]:
empatheticdialogues_data['gemini-1.5-flash'][0].head()

,full_conversation,empathy,relevance,fluency
conversation_id,,,,
hit:401_conv:803,Seeker: The result of my final college exams w...,2,4,5
hit:478_conv:956,Seeker: when i was going for an interview i wa...,4,2,4
hit:887_conv:1775,Seeker: I felt really left out when my friends...,2,2,5
hit:965_conv:1930,Seeker: I saw a shark in the water last week. ...,4,4,5
hit:3379_conv:6758,Seeker: During a work meeting_comma_ I was pre...,3,3,5


In [21]:
def empatheticdialogues_summary(model):
    ranges = pd.DataFrame(columns= ['empathy', 
                                        'relevance', 
                                        'fluency'])

    summary = score_summary(empatheticdialogues_data, 'conversation_id', 2, model)
    cols = summary.columns[1:]
    summary[cols] = summary[cols].apply(pd.to_numeric, errors='coerce')
    summary = summary.describe()[['empathy_range', 
                                  'relevance_range',
                                  'fluency_range']]
    
    summary = summary.rename(columns={'empathy_range': 'empathy', 
                                                    'relevance_range': 'relevance', 
                                                    'fluency_range': 'fluency'})
    summary = summary.loc[['mean', 'std']]

    ranges = pd.concat([summary], axis=1).T

    # Rename the rows
    ranges.rename(columns = {'mean': f'{model}_mean', 'std': f'{model}_std'}, inplace= True)
    ranges = ranges.transpose()
    return ranges

In [22]:
final_empatheticdialogues_summary = empatheticdialogues_summary('gemini-1.5-flash')
final_empatheticdialogues_summary = pd.concat([final_empatheticdialogues_summary, empatheticdialogues_summary('gpt-4o')])
final_empatheticdialogues_summary.to_csv('empatheticdialogues_robustness_check.csv')

In [341]:
final_empatheticdialogues_summary

,empathy,relevance,fluency
gemini-1.5-flash_mean,0.000000,0.000000,0.000000
gemini-1.5-flash_std,0.000000,0.000000,0.000000
gpt-4o_mean,0.100000,0.320000,0.220000
gpt-4o_std,0.303046,0.471212,0.418452
